# EEG-LFP 预处理完整流程

本notebook演示从BIDS格式数据到预处理完成的完整流程。

## 流程概览
1. 数据检查与验证
2. 通用清洗（去趋势、滤波、重采样）
3. EEG预处理（坏导检测、重参考、ICA去伪迹、分段、源重建）
4. LFP预处理（刺激伪迹去除、电极管理、降噪）
5. 联合处理（时间对齐、频段分解、标准化）
6. 质量控制与保存

In [ ]:
# 导入必要的库
import sys
sys.path.append('../src')

import numpy as np
import mne
import matplotlib.pyplot as plt
from pathlib import Path

# 导入预处理模块
from preprocessing.data_validation import DataValidator
from preprocessing.signal_cleaning import EEGCleaner, LFPCleaner
from preprocessing.eeg_preprocessing import EEGPreprocessor
from preprocessing.lfp_preprocessing import LFPPreprocessor
from preprocessing.joint_preprocessing import JointPreprocessor
from preprocessing.quality_control import QualityControl, BIDSDerivativesSaver

# 设置
mne.set_log_level('WARNING')
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)

## 1. 数据检查与验证

In [ ]:
# 设置BIDS根目录
bids_root = '/path/to/your/bids/directory'
subject = 'sub-01'
session = 'ses-01'
task = 'task-rest'
run = 'run-01'  # 可选

# 创建验证器
validator = DataValidator(bids_root)

# 运行完整验证
validation_results = validator.run_full_validation(
    subject=subject,
    session=session,
    task=task,
    run=run
)

# 生成验证报告
report = validator.generate_validation_report()
print(report)

In [ ]:
# 加载数据供后续使用
eeg_raw, eeg_metadata = validator.load_eeg_data(subject, session, task, run)
lfp_raw, lfp_metadata = validator.load_lfp_data(subject, session, task, run)

print(f"\nEEG数据信息:")
print(eeg_raw.info)
print(f"\nLFP数据信息:")
print(lfp_raw.info)

## 2. 通用清洗

In [ ]:
# 创建清洗器
eeg_cleaner = EEGCleaner()
lfp_cleaner = LFPCleaner()

# 保存原始数据用于对比
eeg_raw_orig = eeg_raw.copy()
lfp_raw_orig = lfp_raw.copy()

# 清洗EEG
print("\n=== 清洗EEG ===")
eeg_raw_clean = eeg_cleaner.apply_eeg_cleaning(
    eeg_raw,
    target_sfreq=1000.0,
    line_freq=50.0,  # 欧洲工频，北美用60.0
    l_freq=1.0,
    h_freq=100.0
)

print(eeg_cleaner.get_processing_summary())

In [ ]:
# 清洗LFP
print("\n=== 清洗LFP ===")
lfp_raw_clean = lfp_cleaner.apply_lfp_cleaning(
    lfp_raw,
    target_sfreq=1000.0,
    line_freq=50.0,
    l_freq=1.0,
    h_freq=200.0
)

print(lfp_cleaner.get_processing_summary())

## 3. EEG专用预处理

In [ ]:
# 创建EEG预处理器
eeg_prep = EEGPreprocessor()

# 3.1 检测并插值坏导
print("\n=== 检测坏导 ===")
bad_channels = eeg_prep.detect_bad_channels(
    eeg_raw_clean,
    method='correlation',
    threshold=0.4
)

if bad_channels:
    eeg_raw_clean = eeg_prep.interpolate_bad_channels(
        eeg_raw_clean,
        bad_channels=bad_channels,
        copy=False
    )

In [ ]:
# 3.2 重参考
print("\n=== 设置参考 ===")
eeg_raw_clean = eeg_prep.set_reference(
    eeg_raw_clean,
    ref_type='average',
    copy=False
)

In [ ]:
# 3.3 ICA去伪迹
print("\n=== 运行ICA ===")
eeg_raw_ica, ica = eeg_prep.run_ica(
    eeg_raw_clean,
    n_components=25,
    method='fastica',
    copy=False
)

# 自动检测伪迹成分
artifact_comps = eeg_prep.detect_artifact_components(
    eeg_raw_clean,
    ica,
    detect_eog=True,
    detect_ecg=True
)

# 可视化ICA成分（可选）
# ica.plot_components(picks=list(range(10)))

# 移除伪迹成分
if artifact_comps['all']:
    eeg_raw_clean = eeg_prep.remove_artifacts_ica(
        eeg_raw_clean,
        ica,
        exclude_components=artifact_comps['all'],
        copy=False
    )

In [ ]:
# 3.4 创建epochs
print("\n=== 创建epochs ===")

# 从注释提取事件
events, event_id = mne.events_from_annotations(eeg_raw_clean, verbose=False)

# 创建epochs
eeg_epochs = eeg_prep.create_epochs(
    eeg_raw_clean,
    events=events,
    event_id=event_id,
    tmin=-0.5,
    tmax=1.5,
    baseline=(-0.2, 0),
    reject=dict(eeg=150e-6)
)

print(f"保留 {len(eeg_epochs)} 个epochs")

In [ ]:
# 3.5 源空间重建（可选，需要MRI数据）
# 如果有个体MRI，可以进行源重建
use_source_reconstruction = False  # 设置为True以启用

if use_source_reconstruction:
    print("\n=== 源空间重建 ===")
    
    subjects_dir = '/path/to/freesurfer/subjects'
    subject_mri = 'fsaverage'  # 或个体受试者ID
    
    # 计算源空间
    source_info = eeg_prep.compute_source_space(
        eeg_raw_clean,
        subjects_dir=subjects_dir,
        subject=subject_mri,
        spacing='oct6'
    )
    
    # 计算逆解
    inverse_solution = eeg_prep.compute_inverse_operator(
        eeg_epochs,
        fwd=source_info['fwd'],
        method='dSPM'
    )
    
    # 提取ROI时间序列
    # 需要定义ROI标签
    # labels_dict = {'M1': m1_label, 'SMA': sma_label}
    # roi_timeseries = eeg_prep.extract_roi_timeseries(
    #     inverse_solution['stcs'],
    #     labels_dict,
    #     mode='mean'
    # )

print(eeg_prep.get_processing_summary())

## 4. LFP专用预处理

In [ ]:
# 创建LFP预处理器
lfp_prep = LFPPreprocessor()

# 4.1 解析电极接触点
print("\n=== 解析电极接触点 ===")
electrode_info = lfp_prep.parse_electrode_contacts(lfp_raw_clean)

print(f"\n左侧电极: {electrode_info['left']}")
print(f"右侧电极: {electrode_info['right']}")

In [ ]:
# 4.2 去除刺激伪迹（如果有DBS刺激）
has_stimulation = False  # 如果有刺激，设置为True

if has_stimulation:
    print("\n=== 去除刺激伪迹 ===")
    
    # 提取刺激事件
    stim_events = None  # 需要从数据中提取
    
    lfp_raw_clean = lfp_prep.remove_stimulation_artifacts(
        lfp_raw_clean,
        stim_events=stim_events,
        method='template',
        window=(-0.005, 0.01),
        copy=False
    )

In [ ]:
# 4.3 应用双极参考
print("\n=== 应用双极参考 ===")
lfp_raw_bipolar = lfp_prep.apply_bipolar_reference(
    lfp_raw_clean,
    copy=True  # 保留单极数据
)

In [ ]:
# 4.4 增强信噪比
print("\n=== 增强信噪比 ===")
lfp_raw_enhanced = lfp_prep.enhance_snr(
    lfp_raw_bipolar,
    method='car',
    copy=True
)

In [ ]:
# 4.5 可选：小波去噪或平滑
use_wavelet_denoising = False  # 可选
use_smoothing = False  # 可选

if use_wavelet_denoising:
    print("\n=== 小波去噪 ===")
    lfp_raw_enhanced = lfp_prep.apply_wavelet_denoising(
        lfp_raw_enhanced,
        wavelet='db4',
        level=4,
        copy=False
    )

if use_smoothing:
    print("\n=== 平滑 ===")
    lfp_raw_enhanced = lfp_prep.apply_smoothing(
        lfp_raw_enhanced,
        window_length=11,
        polyorder=3,
        copy=False
    )

print(lfp_prep.get_processing_summary())

In [ ]:
# 4.6 创建LFP epochs
print("\n=== 创建LFP epochs ===")

lfp_epochs = mne.Epochs(
    lfp_raw_enhanced,
    events,
    event_id,
    tmin=-0.5,
    tmax=1.5,
    baseline=(-0.2, 0),
    preload=True,
    verbose=False
)

print(f"保留 {len(lfp_epochs)} 个epochs")

## 5. 联合处理

In [ ]:
# 创建联合处理器
joint_prep = JointPreprocessor()

# 5.1 对齐时间窗口
print("\n=== 对齐时间窗口 ===")
eeg_aligned, lfp_aligned = joint_prep.align_time_windows(
    eeg_raw_clean,
    lfp_raw_enhanced,
    crop_to='shorter'
)

In [ ]:
# 5.2 同步epochs
print("\n=== 同步epochs ===")
eeg_epochs_sync, lfp_epochs_sync = joint_prep.synchronize_epochs(
    eeg_epochs,
    lfp_epochs,
    tolerance=0.001
)

In [ ]:
# 5.3 频段分解
print("\n=== 频段分解 ===")

# 定义感兴趣的频段
freq_bands = {
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 100)
}

# 提取EEG频段
eeg_bands = joint_prep.extract_frequency_bands(
    eeg_aligned,
    bands=freq_bands,
    method='filter'
)

# 提取LFP频段
lfp_bands = joint_prep.extract_frequency_bands(
    lfp_aligned,
    bands=freq_bands,
    method='filter'
)

In [ ]:
# 5.4 计算频段功率
print("\n=== 计算频段功率 ===")

eeg_band_power = joint_prep.compute_band_power(
    eeg_epochs_sync,
    bands=freq_bands,
    method='welch'
)

lfp_band_power = joint_prep.compute_band_power(
    lfp_epochs_sync,
    bands=freq_bands,
    method='welch'
)

# 显示频段功率形状
for band in freq_bands.keys():
    print(f"  EEG {band}: {eeg_band_power[band].shape}")
    print(f"  LFP {band}: {lfp_band_power[band].shape}")

In [ ]:
# 5.5 标准化
print("\n=== 标准化 ===")

# 标准化频段功率
for band in freq_bands.keys():
    eeg_band_power[band] = joint_prep.normalize_signals(
        eeg_band_power[band],
        method='zscore',
        axis=-1
    )
    
    lfp_band_power[band] = joint_prep.normalize_signals(
        lfp_band_power[band],
        method='zscore',
        axis=-1
    )

In [ ]:
# 5.6 准备连接性分析数据
print("\n=== 准备连接性分析数据 ===")

connectivity_data = joint_prep.prepare_connectivity_data(
    eeg_epochs_sync,
    lfp_epochs_sync
)

print(f"连接性数据形状: {connectivity_data['data'].shape}")
print(f"EEG通道数: {connectivity_data['n_eeg']}")
print(f"LFP通道数: {connectivity_data['n_lfp']}")

print(joint_prep.get_processing_summary())

## 6. 质量控制与保存

In [ ]:
# 创建质量控制器
output_dir = './qc_outputs'
qc = QualityControl(output_dir=output_dir)

# 6.1 绘制功率谱对比
print("\n=== 生成质量控制图 ===")

qc.plot_psd_comparison(
    eeg_raw_orig,
    eeg_raw_clean,
    save_path=f'{output_dir}/eeg_psd_comparison.png'
)

qc.plot_psd_comparison(
    lfp_raw_orig,
    lfp_raw_enhanced,
    save_path=f'{output_dir}/lfp_psd_comparison.png'
)

In [ ]:
# 6.2 绘制信号对比
qc.plot_signal_comparison(
    eeg_raw_orig,
    eeg_raw_clean,
    duration=5.0,
    channel_idx=0,
    save_path=f'{output_dir}/eeg_signal_comparison.png'
)

qc.plot_signal_comparison(
    lfp_raw_orig,
    lfp_raw_enhanced,
    duration=5.0,
    channel_idx=0,
    save_path=f'{output_dir}/lfp_signal_comparison.png'
)

In [ ]:
# 6.3 绘制epochs质量
qc.plot_epochs_quality(
    eeg_epochs_sync,
    save_path=f'{output_dir}/eeg_epochs_quality.png'
)

qc.plot_epochs_quality(
    lfp_epochs_sync,
    save_path=f'{output_dir}/lfp_epochs_quality.png'
)

In [ ]:
# 6.4 绘制频段分解
qc.plot_frequency_bands(
    eeg_bands,
    channel_idx=0,
    duration=5.0,
    save_path=f'{output_dir}/eeg_frequency_bands.png'
)

qc.plot_frequency_bands(
    lfp_bands,
    channel_idx=0,
    duration=5.0,
    save_path=f'{output_dir}/lfp_frequency_bands.png'
)

In [ ]:
# 6.5 计算信噪比
print("\n=== 计算信噪比 ===")

eeg_snr = qc.compute_snr(eeg_raw_clean)
lfp_snr = qc.compute_snr(lfp_raw_enhanced)

In [ ]:
# 6.6 生成质量报告
print("\n=== 生成质量报告 ===")

# 收集所有处理步骤
all_processing_steps = (
    eeg_cleaner.processing_history +
    eeg_prep.processing_log +
    lfp_cleaner.processing_history +
    lfp_prep.processing_log +
    joint_prep.processing_log
)

# 收集质量指标
quality_metrics = {
    'n_eeg_epochs': len(eeg_epochs_sync),
    'n_lfp_epochs': len(lfp_epochs_sync),
    'eeg_mean_snr': np.mean(list(eeg_snr.values())),
    'lfp_mean_snr': np.mean(list(lfp_snr.values())),
    'n_bad_channels': len(bad_channels)
}

# 生成报告
qc_report = qc.generate_qc_report(
    preprocessing_steps=all_processing_steps,
    metrics=quality_metrics,
    save_path=f'{output_dir}/quality_control_report.txt'
)

print(qc_report)

In [ ]:
# 6.7 保存为BIDS derivatives
print("\n=== 保存BIDS derivatives ===")

saver = BIDSDerivativesSaver(
    bids_root=bids_root,
    derivatives_name='preprocessing'
)

# 保存预处理后的原始数据
saver.save_preprocessed_raw(
    eeg_raw_clean,
    subject=subject,
    session=session,
    task=task,
    datatype='eeg',
    suffix='eeg',
    run=run,
    description='clean'
)

saver.save_preprocessed_raw(
    lfp_raw_enhanced,
    subject=subject,
    session=session,
    task=task,
    datatype='ieeg',
    suffix='ieeg',
    run=run,
    description='clean'
)

# 保存epochs
saver.save_epochs(
    eeg_epochs_sync,
    subject=subject,
    session=session,
    task=task,
    datatype='eeg',
    run=run,
    description='clean'
)

saver.save_epochs(
    lfp_epochs_sync,
    subject=subject,
    session=session,
    task=task,
    datatype='ieeg',
    run=run,
    description='clean'
)

# 保存处理元数据
processing_info = {
    'preprocessing_steps': all_processing_steps,
    'quality_metrics': quality_metrics,
    'bad_channels': bad_channels,
    'artifact_components': artifact_comps['all'],
    'frequency_bands': freq_bands
}

saver.save_derivative_metadata(
    processing_info,
    subject=subject,
    session=session
)

print("\n✓ 预处理完成！所有结果已保存。")

## 总结

预处理流程已完成，包括：

1. ✓ 数据验证与检查
2. ✓ 通用清洗（去趋势、滤波、重采样）
3. ✓ EEG预处理（坏导插值、重参考、ICA、分段）
4. ✓ LFP预处理（伪迹去除、双极参考、增强SNR）
5. ✓ 联合处理（时间对齐、频段分解、标准化）
6. ✓ 质量控制与保存

下一步可以进行：
- 跨通道连接性分析
- 时频分析
- 相位-振幅耦合（PAC）
- 功能性连接
- 统计分析